In [1]:
import json
from time import sleep
import re
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

## archive

In [2]:
df_all=pd.read_csv('no_dup_dog.csv')

In [3]:
df_all

,Unnamed: 0,AnimalName,AnimalGender,AnimalBirthYear,BreedName,ZipCode,LicenseIssuedDate,LicenseExpiredDate,Extract Year
0,0,PAIGE,F,2014,american pit bull mix / pit bull mix,10035.0,09/12/2014,09/12/2017,2016
1,2,ALI,M,2014,basenji,10013.0,09/12/2014,09/12/2019,2016
2,5,IAN,M,2006,unknown,10013.0,09/12/2014,10/30/2019,2016
3,11,LEMMY,F,2005,yorkshire terrier,10003.0,09/13/2014,10/26/2017,2016
4,20,OTIS,F,2001,unknown,10011.0,09/13/2014,09/29/2017,2016
...,...,...,...,...,...,...,...,...,...
213078,508191,PRINCE,M,2016,samoyed,11221.0,01/23/2020,01/23/2022,2022
213079,508192,UNKNOWN,F,2019,cavalier king charles spaniel,10025.0,01/23/2020,01/23/2021,2022
213080,508193,FINBAR,M,2016,"poodle, standard",10023.0,01/23/2020,02/27/2021,2022
213081,508194,UNKNOWN,M,2019,cockapoo,11378.0,01/23/2020,01/23/2021,2022


In [5]:
# df_index=df_all.set_index('dup')

In [6]:
df_clean=df_all.drop(['Unnamed: 0'], axis=1)

In [7]:
df_clean

,AnimalName,AnimalGender,AnimalBirthYear,BreedName,ZipCode,LicenseIssuedDate,LicenseExpiredDate,Extract Year
0,PAIGE,F,2014,american pit bull mix / pit bull mix,10035.0,09/12/2014,09/12/2017,2016
1,ALI,M,2014,basenji,10013.0,09/12/2014,09/12/2019,2016
2,IAN,M,2006,unknown,10013.0,09/12/2014,10/30/2019,2016
3,LEMMY,F,2005,yorkshire terrier,10003.0,09/13/2014,10/26/2017,2016
4,OTIS,F,2001,unknown,10011.0,09/13/2014,09/29/2017,2016
...,...,...,...,...,...,...,...,...
213078,PRINCE,M,2016,samoyed,11221.0,01/23/2020,01/23/2022,2022
213079,UNKNOWN,F,2019,cavalier king charles spaniel,10025.0,01/23/2020,01/23/2021,2022
213080,FINBAR,M,2016,"poodle, standard",10023.0,01/23/2020,02/27/2021,2022
213081,UNKNOWN,M,2019,cockapoo,11378.0,01/23/2020,01/23/2021,2022


In [49]:
info_df=pd.read_csv('dog_breed_in.csv')


In [50]:
info_df['height'] = info_df['height'].str.replace('[', '')
info_df['height'] = info_df['height'].str.replace(']', '')
# info_df['height'].astype(int)

/var/folders/7n/dgbss8317f56vp5jbzxg8m2m0000gn/T/ipykernel_76265/4086843079.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  info_df['height'] = info_df['height'].str.replace('[', '')
/var/folders/7n/dgbss8317f56vp5jbzxg8m2m0000gn/T/ipykernel_76265/4086843079.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  info_df['height'] = info_df['height'].str.replace(']', '')


In [51]:
info_df['weight'] = info_df['weight'].str.replace('[', '')
info_df['weight'] = info_df['weight'].str.replace(']', '')

/var/folders/7n/dgbss8317f56vp5jbzxg8m2m0000gn/T/ipykernel_76265/3763323086.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  info_df['weight'] = info_df['weight'].str.replace('[', '')
/var/folders/7n/dgbss8317f56vp5jbzxg8m2m0000gn/T/ipykernel_76265/3763323086.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  info_df['weight'] = info_df['weight'].str.replace(']', '')


In [58]:
info_df['weight'] = info_df['weight'].str[-2:]
info_df['height'] = info_df['height'].str[-2:]

In [59]:
info_df

,Unnamed: 0,breed,url,height,weight,longivity
0,0,afador,https://dogtime.com/dog-breeds/afador,29,75,[12]
1,1,afaird,https://dogtime.com/dog-breeds/afaird-dog-bree...,24,80,[12]
2,2,affen border terrier,https://dogtime.com/dog-breeds/affen-border-te...,16,14,[14]
3,3,affen spaniel,https://dogtime.com/dog-breeds/affen-spaniel,20,20,[15]
4,4,affen tzu,https://dogtime.com/dog-breeds/affen-tzu-dog-b...,11,,[15]
...,...,...,...,...,...,...
471,471,working kelpie,https://dogtime.com/dog-breeds/working-kelpie,25,60,[15]
472,472,xoloitzcuintli,https://dogtime.com/dog-breeds/xoloitzuintli,11,50,[20]
473,473,yakutian laika,https://dogtime.com/dog-breeds/yakutian-laika,23,55,[12]
474,474,yorkipoo,https://dogtime.com/dog-breeds/yorkipoo,15,14,[15]


In [27]:
# info_df['height']=info_df['height'].str.isdigit().mean()

In [1]:
# df_new=df_new.drop(['LicenseIssuedDate','LicenseExpiredDate','Extract Year'],axis=1)

In [23]:
# df_clean['BreedName'] = df_clean['BreedName'].str.lower()

In [60]:
info_df.columns=['Unnamed: 0', 'BreedName', 'url', 'height', 'weight', 'longivity']

In [61]:
# df_all = pd.merge(df, info_df, on=['BreedName'])
df_new=df_clean.set_index('BreedName').join(info_df.set_index('BreedName'))


In [63]:
df_new.drop(['Unnamed: 0'], axis=1)

,AnimalName,AnimalGender,AnimalBirthYear,ZipCode,LicenseIssuedDate,LicenseExpiredDate,Extract Year,url,height,weight,longivity
BreedName,,,,,,,,,,,
6152012,NELLIE,F,2011,10014.0,05/31/2021,07/01/2022,2022,NaN,NaN,NaN,NaN
affenpinscher,OZZY,M,2013,10038.0,01/19/2015,01/19/2016,2016,https://dogtime.com/dog-breeds/affenpinscher,11,9,[14]
affenpinscher,CHEWBACCA,M,2011,10065.0,02/04/2015,02/04/2017,2016,https://dogtime.com/dog-breeds/affenpinscher,11,9,[14]
affenpinscher,DORADO,M,2013,10038.0,02/27/2015,02/27/2016,2016,https://dogtime.com/dog-breeds/affenpinscher,11,9,[14]
affenpinscher,REX,M,2000,11237.0,04/25/2015,04/25/2016,2016,https://dogtime.com/dog-breeds/affenpinscher,11,9,[14]
...,...,...,...,...,...,...,...,...,...,...,...
zuchon,MIKAYLA,F,2010,10314.0,05/24/2021,07/07/2022,2022,NaN,NaN,NaN,NaN
zuchon,SUNNY,F,2013,10312.0,06/17/2021,04/02/2022,2022,NaN,NaN,NaN,NaN
zuchon,MAX,M,2011,10314.0,11/05/2021,09/20/2022,2022,NaN,NaN,NaN,NaN


In [34]:
df_no_dup_dog_info=df_new.reset_index(drop=False)

## export to csv

In [64]:
# df_new.to_csv('dog_info_dec_3.csv')

In [46]:
# df_final.to_csv('info_final.csv')

## real

In [10]:
# df=pd.read_csv('has_info_gos.csv')

In [14]:
# df.sort_values(by='AnimalName')

,Unnamed: 0.1,AnimalName,AnimalGender,AnimalBirthYear,BreedName,ZipCode,Unnamed: 0,url,height,weight,longivity
61288,61288,'RUSTY,M,2014,cavalier king charles spaniel,11378.0,177,https://dogtime.com/dog-breeds/cavalier-king-c...,[13],[18],[15]
62281,62281,'RUSTY,M,2014,cavalier king charles spaniel,11378.0,177,https://dogtime.com/dog-breeds/cavalier-king-c...,[13],[18],[15]
63003,63003,'RUSTY,M,2014,cavalier king charles spaniel,11378.0,177,https://dogtime.com/dog-breeds/cavalier-king-c...,[13],[18],[15]
163278,163278,.,M,2020,shih tzu,10024.0,421,https://dogtime.com/dog-breeds/shih-tzu,[10],[16],[16]
297059,297059,.,M,2019,goldendoodle,11421.0,269,https://dogtime.com/dog-breeds/goldendoodle,[24],[90],[15]
...,...,...,...,...,...,...,...,...,...,...,...
295056,295056,NaN,M,2016,goldendoodle,11385.0,269,https://dogtime.com/dog-breeds/goldendoodle,[24],[90],[15]
295529,295529,NaN,M,2016,goldendoodle,11385.0,269,https://dogtime.com/dog-breeds/goldendoodle,[24],[90],[15]
295913,295913,NaN,F,2018,goldendoodle,10472.0,269,https://dogtime.com/dog-breeds/goldendoodle,[24],[90],[15]
296073,296073,NaN,M,2016,goldendoodle,11385.0,269,https://dogtime.com/dog-breeds/goldendoodle,[24],[90],[15]


In [6]:
df_all['BreedName'] = df_all['BreedName'].str.lower()

In [10]:
df_all

,dup,AnimalName,AnimalGender,AnimalBirthYear,BreedName,ZipCode,LicenseIssuedDate,LicenseExpiredDate,Extract Year
0,0,PAIGE,F,2014,american pit bull mix / pit bull mix,10035.0,09/12/2014,09/12/2017,2016
1,0,YOGI,M,2010,boxer,10465.0,09/12/2014,10/02/2017,2016
2,0,ALI,M,2014,basenji,10013.0,09/12/2014,09/12/2019,2016
3,0,QUEEN,F,2013,akita crossbreed,10013.0,09/12/2014,09/12/2017,2016
4,0,LOLA,F,2009,maltese,10028.0,09/12/2014,10/09/2017,2016
...,...,...,...,...,...,...,...,...,...
508191,1,PRINCE,M,2016,samoyed,11221.0,01/23/2020,01/23/2022,2022
508192,1,UNKNOWN,F,2019,cavalier king charles spaniel,10025.0,01/23/2020,01/23/2021,2022
508193,1,FINBAR,M,2016,"poodle, standard",10023.0,01/23/2020,02/27/2021,2022
508194,1,UNKNOWN,M,2019,cockapoo,11378.0,01/23/2020,01/23/2021,2022


#### dog food https://www.petmd.com/blogs/nutritionnuggets/dr-coates/2015/july/are-you-feeding-your-dog-right-amount-32905
